In [50]:
import requests, json
from urllib.parse import quote
import folium
import pandas as pd
import numpy as np

In [51]:
filename = 'data/경기도_주민등록인구집계현황.csv'
pop = pd.read_csv(filename, encoding='euc-kr')
pop.head()

,연도,월,행정구역구분명,행정구역명,총 인구수,0~9세,10~19세,20~29세,30~39세,40~49세,...,10~19세 (여),20~29세 (여),30~39세 (여),40~49세 (여),50~59세 (여),60~69세 (여),70~79세 (여),80~89세 (여),90~99세 (여),100세 이상 (여)
0,2022,12,도,경기도,13589432,1040954,1327726,1737291,1882300,2284603,...,647283,821150,904073,1121889,1133428,878494,437251,252711,44184,1369
1,2022,12,시군,경기도 가평군,62197,2930,4463,5929,5397,7589,...,2186,2503,2433,3525,5090,6439,3653,2408,411,13
2,2022,12,읍면동,경기도 가평군 가평읍,19555,1020,1673,2094,1795,2630,...,778,937,848,1244,1583,1805,1054,739,126,6
3,2022,12,읍면동,경기도 가평군 북면,3815,108,174,230,223,357,...,93,87,99,154,290,511,294,216,39,1
4,2022,12,읍면동,경기도 가평군 상면,5669,140,259,482,389,533,...,112,151,166,240,515,737,389,244,39,0


In [52]:
pop = pop[['행정구역구분명','행정구역명','총 인구수']]
pop.head()

,행정구역구분명,행정구역명,총 인구수
0,도,경기도,13589432
1,시군,경기도 가평군,62197
2,읍면동,경기도 가평군 가평읍,19555
3,읍면동,경기도 가평군 북면,3815
4,읍면동,경기도 가평군 상면,5669


In [53]:
pop.drop(0, inplace=True)
pop.head()

,행정구역구분명,행정구역명,총 인구수
1,시군,경기도 가평군,62197
2,읍면동,경기도 가평군 가평읍,19555
3,읍면동,경기도 가평군 북면,3815
4,읍면동,경기도 가평군 상면,5669
5,읍면동,경기도 가평군 설악면,9679


In [54]:
pop = pop[pop.행정구역구분명.str.contains('시군') | pop.행정구역구분명.str.contains('구')]
pop.head()

,행정구역구분명,행정구역명,총 인구수
1,시군,경기도 가평군,62197
8,시군,경기도 고양시,1075202
9,구,경기도 고양시 덕양구,487874
31,구,경기도 고양시 일산동구,296590
44,구,경기도 고양시 일산서구,290738


In [55]:
gu_city = ['수원시', '고양시', '용인시', '성남시', '안산시', '안양시']
pop.행정구역명 = pop.행정구역명.str.strip()
drop_index = []
for index in pop[pop.행정구역구분명 == '시군'].index:
    if pop.행정구역명[index].split()[-1] in gu_city:
        drop_index.append(index)
pop.drop(drop_index, inplace=True)
pop.head()

,행정구역구분명,행정구역명,총 인구수
1,시군,경기도 가평군,62197
9,구,경기도 고양시 덕양구,487874
31,구,경기도 고양시 일산동구,296590
44,구,경기도 고양시 일산서구,290738
56,시군,경기도 과천시,77775


In [56]:
filename = 'data/경기도_geo_simple.json'
geo_data = json.load(open(filename, encoding='utf-8'))

In [57]:
area = pd.read_csv('data/전국도시공원정보표준데이터.csv',encoding='euc-kr')
area.head()

,관리번호,공원명,공원구분,소재지도로명주소,소재지지번주소,위도,경도,공원면적,공원보유시설(운동시설),공원보유시설(유희시설),공원보유시설(편익시설),공원보유시설(교양시설),공원보유시설(기타시설),지정고시일,관리기관명,전화번호,데이터기준일자,제공기관코드,제공기관명
0,11170-00048,녹색소공원,소공원,NaN,서울특별시 용산구 효창동 287-6,37.539217,126.962694,1569.9,NaN,NaN,NaN,NaN,NaN,2012-10-19,서울특별시 용산구청,02-2199-7605,2022-12-20,3020000,서울특별시 용산구
1,36110-00058,푸름샘 소공원(소1-1),소공원,세종특별자치시 마음안로 161,NaN,36.516117,127.233781,1820.0,NaN,NaN,앉음벽,NaN,실개천,2011-08-05,세종특별자치시 공원관리사업소,044-301-3613,2022-12-14,5690000,세종특별자치시
2,36110-00059,솔숲 소공원(소1-2),소공원,세종특별자치시 만남1북로 73,NaN,36.508995,127.230040,1037.0,NaN,조합놀이대+ 그네+ 시소+ 흔들놀이,의자+ 파고라,NaN,안내판,2011-08-05,세종특별자치시 공원관리사업소,044-301-3613,2022-12-14,5690000,세종특별자치시
3,36110-00060,마디꽃 수변공원(수1-1),수변공원,세종특별자치시 다솜로 38,NaN,36.495962,127.257316,4928.0,NaN,NaN,정자+ 의자,NaN,NaN,2007-07-04,세종특별자치시 공원관리사업소,044-301-3633,2022-12-14,5690000,세종특별자치시
4,36110-00061,꽃 수변공원(수2-3),수변공원,세종특별자치시 나리로 26-10,NaN,36.473133,127.254482,3575.0,배드민터장+ 운동기구3종,NaN,자전거보관대+ 평의자+ 안내판,NaN,NaN,2009-01-07,세종특별자치시 공원관리사업소,044-301-3613,2022-12-14,5690000,세종특별자치시


In [58]:
area = area[['공원명','소재지지번주소','공원면적']]

In [59]:
area.dropna(subset=['소재지지번주소'], inplace=True)
area.isna().sum().sum()

0

In [60]:
area.head()

,공원명,소재지지번주소,공원면적
0,녹색소공원,서울특별시 용산구 효창동 287-6,1569.9
23,강변공원,서울특별시 용산구 이촌동 408,1766.0
24,솔밭어린이공원,서울특별시 용산구 한강로1가 30-3,2090.0
25,신창동어린이공원,서울특별시 용산구 신창동 100,257.0
26,서빙고공원,서울특별시 용산구 용산동5가 24-2,11879.0


In [61]:
area = area[area['소재지지번주소'].str.contains('경기도')]
area.head()

,공원명,소재지지번주소,공원면적
212,꿈나무공원,경기도 이천시 중리동 15,2526.0
213,남천공원,경기도 이천시 중리동 210,3830.0
214,향교공원,경기도 이천시 창전동 408,1757.0
215,개나리공원,경기도 이천시 창전동 414-13,1650.0
216,달맞이공원,경기도 이천시 창전동 423,1650.0


In [62]:
area['시군구'] = area.소재지지번주소.apply(lambda x: ''.join(x.split()[1]))
area.head()

,공원명,소재지지번주소,공원면적,시군구
212,꿈나무공원,경기도 이천시 중리동 15,2526.0,이천시
213,남천공원,경기도 이천시 중리동 210,3830.0,이천시
214,향교공원,경기도 이천시 창전동 408,1757.0,이천시
215,개나리공원,경기도 이천시 창전동 414-13,1650.0,이천시
216,달맞이공원,경기도 이천시 창전동 423,1650.0,이천시


In [69]:
pop['시군구'] = pop.행정구역명.apply(lambda x: ''.join(x.split()[1]))
pop.head()

,행정구역구분명,행정구역명,총 인구수,시군구
시군구,,,,
가평군,시군,경기도 가평군,62197,가평군
고양시,구,경기도 고양시 덕양구,487874,고양시
고양시,구,경기도 고양시 일산동구,296590,고양시
고양시,구,경기도 고양시 일산서구,290738,고양시
과천시,시군,경기도 과천시,77775,과천시


In [64]:
pop.set_index('시군구', inplace=True)
pop.head()

,행정구역구분명,행정구역명,총 인구수
시군구,,,
가평군,시군,경기도 가평군,62197
고양시,구,경기도 고양시 덕양구,487874
고양시,구,경기도 고양시 일산동구,296590
고양시,구,경기도 고양시 일산서구,290738
과천시,시군,경기도 과천시,77775


In [65]:
sort_area = area[['공원면적','시군구']].groupby('시군구').sum()
sort_area

,공원면적
시군구,
가평군,511674.5
고양시,6932758.3
과천시,315666.0
광명시,3397531.0
광주시,786484.6
구리시,912756.3
군포시,1455419.2
김포시,5792950.0
남양주시,4770408.0


In [66]:
sort_pop=pop.sort_index()
sort_pop

,행정구역구분명,행정구역명,총 인구수
시군구,,,
가평군,시군,경기도 가평군,62197
고양시,구,경기도 고양시 덕양구,487874
고양시,구,경기도 고양시 일산동구,296590
고양시,구,경기도 고양시 일산서구,290738
과천시,시군,경기도 과천시,77775
광명시,시군,경기도 광명시,288182
광주시,시군,경기도 광주시,391704
광주시,시군,경기도 광주시 오포2동,34293
구리시,시군,경기도 구리시,188876


In [67]:
pop_area = pd.merge(left = sort_pop , right = sort_area, how = "right", on = "자치구")
pop_area

KeyError: '자치구'